In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mostrar el promedio de ratings para las apps de cada categoría de tipo pagas o gratuitas teniendo en cuenta solo a las apps que tengan al menos una review de mas de 20 palabras o alguna que incluya ‘good’/’bad’. (⭐⭐)

In [2]:
import pandas as pd
def palabras(texto):
  palabras = 0
  for palabra in texto.split():
    palabras += 1
  return palabras

In [3]:
import pandas as pd
path_google_playstore = '/content/drive/MyDrive/Organizacion de datos/TP1 - Pandas, Spark y Visualización de datos/googleplaystore.csv'
path_reviews = '/content/drive/MyDrive/Organizacion de datos/TP1 - Pandas, Spark y Visualización de datos/googleplaystore_user_reviews.csv'

apps_df = pd.read_csv(path_google_playstore)
reviews_df = pd.read_csv(path_reviews)

Elimino todas las filas que tengan valor nulo en la review (no tendran mas de 20 palabras ni la palabra 'good'/'bad')

In [4]:
filtered_reviews_df = reviews_df.loc[~reviews_df['Translated_Review'].isnull(), :]

Considero la busqueda de 'good'/'bad' en una review, sin distinguir mayusculas de minusculas


In [5]:
filtered_reviews_df = filtered_reviews_df.loc[filtered_reviews_df['Translated_Review'].apply(lambda review: len(review.split()) > 20 | ("good"in review.lower() or "bad" in review.lower())), :]
apps_and_reviews_df = pd.merge(apps_df, filtered_reviews_df, on='App')

Una vez aplicado el filtro elimino todas las apps repetidas del df: considero una app repetida si tiene exactaemente el mismo nombre, cantidad de reviews y tamaño que otra.

In [6]:
apps_and_reviews_df_clean = apps_and_reviews_df.drop_duplicates(subset=['App', 'Reviews', 'Size'])

Elimino las filas que tienen valor NaN en la columna Rating (no aportan al promedio)


In [7]:
apps_and_reviews_df.dropna(subset=['Rating'], inplace=True)

Elimino apps con valores incorrectos (el promedio va a cambiar si un valor no corresponde)


In [8]:
apps_and_reviews_df = apps_and_reviews_df[(apps_and_reviews_df['Rating'] >= 0) & (apps_and_reviews_df['Rating'] <= 5)]

Elimino aquellas que no tengan un valor valido en 'Type'


In [9]:
apps_and_reviews_df = apps_and_reviews_df[apps_and_reviews_df['Type'].isin(['Paid', 'Free'])]

Si una categoria en un tipo no tiene niguna aplicacion que cumpla con todas las condiciones pedidas, no se muestra en el df el promedio para esa categoria y ese tipo

In [10]:
promedio_ratings_cat = apps_and_reviews_df.groupby(['Category', 'Type'])['Rating'].mean().reset_index(name='Average Rating')
promedio_ratings_cat_free = promedio_ratings_cat.loc[promedio_ratings_cat['Type'] == 'Free', :].reset_index(drop=True)
promedio_ratings_cat_paid = promedio_ratings_cat.loc[promedio_ratings_cat['Type'] == 'Paid', :].reset_index(drop=True)
promedio_ratings_cat_free.drop(columns='Type', inplace=True)
promedio_ratings_cat_paid.drop(columns='Type', inplace=True)

In [11]:
display(promedio_ratings_cat_paid.style.set_caption("Paid apps average rating by category (with apps with at least one review with more than 20 or the word 'good'/'bad')"))
display(promedio_ratings_cat_free.style.set_caption("Free apps average rating by category (with apps with at least one review with more than 20 or the word 'good'/'bad')"))

,Category,Average Rating
0,FAMILY,4.400000
1,GAME,4.460870
2,MEDICAL,4.260000
3,PERSONALIZATION,4.251678
4,SPORTS,4.100000


,Category,Average Rating
0,ART_AND_DESIGN,4.546875
1,AUTO_AND_VEHICLES,4.497059
2,BEAUTY,4.063636
3,BOOKS_AND_REFERENCE,4.342276
4,BUSINESS,4.229477
5,COMICS,4.200000
6,COMMUNICATION,4.257572
7,DATING,4.014781
8,EDUCATION,4.616447
9,ENTERTAINMENT,3.932987
